In [1]:
pip install -U keras-tuner

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf
print(tf.__version__)


2023-09-25 06:52:17.295668: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-25 06:52:18.327320: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-25 06:52:18.331008: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-25 06:52:20.937852: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


2.13.0


In [3]:
mnist = tf.keras.datasets.mnist

In [4]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train, X_test = X_train/255, X_test/255
print(X_train.shape)

(60000, 28, 28)


In [5]:
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner.tuners import RandomSearch

Using TensorFlow backend


In [6]:
def build_model(hp):
    model = keras.Sequential()
    
    model.add(layers.Flatten(input_shape = (28,28)))
    
    model.add(layers.Dense(units = hp.Int('units', 
                                          min_value = 32,
                                          max_value= 512,
                                          step = 128),
                                          activation = 'relu'))
    
    model.add(layers.Dense (10, activation = 'softmax'))

    model.compile(
        optimizer = keras.optimizers.Adam(
            hp.Choice('learning_rate',
                     values = [.01, .001])),
        loss = 'sparse_categorical_crossentropy',
        metrics = ['accuracy'])
    
    return model


    

In [7]:
tuner = RandomSearch(
    build_model,
    objective = 'val_accuracy',
    max_trials = 5,
    executions_per_trial = 3,
    directory = 'my_dir',
    project_name = 'helloworld'
)

Reloading Tuner from my_dir/helloworld/tuner0.json


In [8]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 128, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [9]:
tuner.search(X_train, y_train, epochs = 5, validation_data = (X_test, y_test))

Trial 5 Complete [00h 01m 34s]
val_accuracy: 0.962399979432424

Best val_accuracy So Far: 0.962399979432424
Total elapsed time: 00h 05m 06s


In [10]:
tuner.results_summary()

Results summary
Results in my_dir/helloworld
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 4 summary
Hyperparameters:
units: 288
learning_rate: 0.01
Score: 0.962399979432424

Trial 2 summary
Hyperparameters:
units: 416
learning_rate: 0.01
Score: 0.9623333414395651

Trial 3 summary
Hyperparameters:
units: 32
learning_rate: 0.0001
Score: 0.928166667620341

Trial 0 summary
Hyperparameters:
units: 160
learning_rate: 0.01
Traceback (most recent call last):
  File "/home/m1069009/.local/lib/python3.8/site-packages/keras_tuner/src/engine/base_tuner.py", line 268, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/home/m1069009/.local/lib/python3.8/site-packages/keras_tuner/src/engine/base_tuner.py", line 233, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/home/m1069009/.local/lib/python3.8/site-packages/keras_tuner/src/engine/tuner.py", line 307, in run_trial
    o

In [11]:
def build_model2(hp):
    model = tf.keras.Sequential()
    model.add(layers.Flatten(input_shape = (28, 28)))
    for i in range (hp.Int('layers', 2, 6)):
        model.add(tf.keras.layers.Dense(
            units = hp.Int('units_'+str(i), 50,100, step = 10),
            activation = hp.Choice('act_'+str(i), ['relu', 'sigmoid'])))
    model.add(tf.keras.layers.Dense(10, activation = 'softmax'))
    model.compile('adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    return model
    


In [12]:
tuner2 = RandomSearch(
    build_model2,
    objective = 'val_accuracy',
    max_trials = 4,
    executions_per_trial = 1,
    directory = 'my_dir2',
    project_name = 'helloworld2'
)

In [13]:
tuner2.search_space_summary()

Search space summary
Default search space size: 5
layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 6, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 50, 'max_value': 100, 'step': 10, 'sampling': 'linear'}
act_0 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid'], 'ordered': False}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 50, 'max_value': 100, 'step': 10, 'sampling': 'linear'}
act_1 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid'], 'ordered': False}


In [14]:
tuner2.search(X_train, y_train, epochs = 5, validation_data = (X_test, y_test))

Trial 4 Complete [00h 00m 17s]
val_accuracy: 0.9750000238418579

Best val_accuracy So Far: 0.9750000238418579
Total elapsed time: 00h 01m 13s
